In [ ]:
import os
import sys
#from typing import Union, Any, Optional, Callable, Tuple

ROOT_DIR = os.path.dirname(os.path.dirname(os.getcwd()))
if ROOT_DIR not in sys.path: sys.path.append(ROOT_DIR)

import numpy as np
#import pandas as pd
import proplot as plot
#from scipy.stats import pearsonr as linear_correlation
import tensorflow as tf
#import eagerpy as ep
#import torch
#from torch import nn, optim
#from torch.nn import functional as F

#from DeepSparseCoding.tf1x.utils.logger import Logger as tfLogger
import DeepSparseCoding.tf1x.analysis.analysis_picker as ap
#from DeepSparseCoding.tf1x.data.dataset import Dataset
import DeepSparseCoding.tf1x.data.data_selector as ds
#import DeepSparseCoding.tf1x.utils.data_processing as tfdp
import DeepSparseCoding.tf1x.params.param_picker as pp
import DeepSparseCoding.tf1x.models.model_picker as mp

#from DeepSparseCoding.utils.file_utils import Logger
#import DeepSparseCoding.utils.dataset_utils as dataset_utils
#import DeepSparseCoding.utils.loaders as loaders
import DeepSparseCoding.utils.plot_functions as pf

#import foolbox
#from foolbox import PyTorchModel
#from foolbox.attacks.projected_gradient_descent import LinfProjectedGradientDescentAttack
#from foolbox.types import Bounds
#from foolbox.models.base import Model
#from foolbox.attacks.base import T
#from foolbox.criteria import Misclassification
#from foolbox.attacks.base import raise_if_kwargs
#from foolbox.attacks.base import get_criterion

rand_seed = 123
rand_state = np.random.RandomState(rand_seed)

### Load DeepSparseCoding analyzer

In [ ]:
class params(object):
  def __init__(self):
    self.device = "/gpu:0"
    self.analysis_dataset = "test"
    self.save_info = "analysis_" + self.analysis_dataset
    self.overwrite_analysis_log = False
    self.do_class_adversaries = False
    self.do_run_analysis = False
    self.do_evals = False
    self.do_basis_analysis = False
    self.do_inference = False
    self.do_atas = False 
    self.do_recon_adversaries = False
    self.do_neuron_visualization = False
    self.do_full_recon = False
    self.do_orientation_analysis = False 
    self.do_group_recons = False
    
    self.data_dir = os.path.join(ROOT_DIR, 'Datasets')
    self.data_type = 'vanhateren'
    self.vectorize_data = True
    self.rescale_data = False
    self.standardize_data = False
    self.contrast_normalize = False
    self.whiten_data = True
    self.whiten_method = "FT"
    self.whiten_batch_size = 2
    self.extract_patches = True
    self.num_patches = 1e4
    self.patch_edge_size = 16
    self.overlapping_patches = True
    self.randomize_patches = True
    self.patch_variance_threshold = 0.0
    self.lpf_data = False # whitening automatically includes lpf
    self.lpf_cutoff = 0.7
    self.batch_size = 100
    self.random_seed = rand_seed

In [ ]:
analysis_params = params()
analysis_params.projects_dir = os.path.expanduser("~")+"/Work/Projects/"

model_names = ['lca_512_vh', 'lca_1024_vh', 'lca_2560_vh']#, 'sae_768_vh', 'rica_768_vh']
model_types = ['LCA', 'LCA', 'LCA']#, 'SAE', 'ICA']
model_labels = ['2x', '4x', '10x']#, 'Sparse Autoencoder', 'Linear Autoencoder']
analyzers = []
for model_type, model_name, model_label in zip(model_types, model_names, model_labels):
    analysis_params.model_name = model_name
    analysis_params.version = '0.0'
    analysis_params.model_dir = analysis_params.projects_dir+analysis_params.model_name
    model_log_file = (analysis_params.model_dir+"/logfiles/"+analysis_params.model_name
      +"_v"+analysis_params.version+".log")
    analysis_params.model_type = model_type
    analyzer = ap.get_analyzer(analysis_params.model_type)
    analysis_params.save_info = "analysis_selectivity"
    analyzer.setup(analysis_params)
    analyzer.model_label = model_label
    analyzer.model_type = model_type
    analyzer.setup_model(analyzer.model_params)
    analyzers.append(analyzer)

In [ ]:
data = ds.get_data(analysis_params)
data = analyzers[0].model.preprocess_dataset(data, analysis_params)
data = analyzers[0].model.reshape_dataset(data, analysis_params)

### VH data examples

In [ ]:
num_imgs = 6
img_idx = np.random.randint(analysis_params.batch_size)
fig, axs = plot.subplots(ncols=6)
for inc_img in range(num_imgs):
    im = axs[inc_img].imshow(data['train'].images[img_idx+inc_img,...].reshape(16, 16), cmap='greys_r')
axs.format(suptitle=f'DSC van hateren example')
pf.clear_axes(axs)
plot.show()

In [ ]:
weights = [np.squeeze(analyzer.eval_analysis(data['train'].images[0,...][None,...], ['lca/weights/w:0'], analyzer.analysis_params.save_info)['lca/weights/w:0']) for analyzer in analyzers if analyzer.model_type=='LCA']

In [ ]:
activations = [np.squeeze(analyzer.compute_activations(data['train'].images[0:100,...],
    activation_operation=analyzer.model.get_encodings))
    for analyzer in analyzers]

In [ ]:
  def compute_lambda_activations(images, model, weights, batch_size=None, activation_operation=None):
    """
    Computes the output code for a set of images.
    Outputs:
      evaluated activation_operation on the input images
    Inputs:
      images [np.ndarray] of shape (num_imgs, num_img_pixels)
      batch_size [int] how many inputs to use in a batch
      activation_operation [tf operation] that produces the output activation
        if None then it defaults to `model.get_encodings()`
    """
    if activation_operation is None:
        activation_operation = model.get_encodings
    images_shape = list(images.shape)
    num_images = images_shape[0]
    config = tf.compat.v1.ConfigProto()
    config.gpu_options.allow_growth = True
    with tf.compat.v1.Session(config=config, graph=model.graph) as sess:
        if batch_size is not None and batch_size < num_images:
            assert num_images % batch_size == 0, (
                "batch_size=%g must divide evenly into num_images=%g"%(batch_size, num_images))
            num_batches = int(np.ceil(num_images / batch_size))
            batch_image_shape = [batch_size] + images_shape[1:]
            sess.run(model.init_op, {model.input_placeholder:np.zeros(batch_image_shape)})
            activations = []
            for batch_idx in range(num_batches):
                im_batch_start_idx = int(batch_idx * batch_size)
                im_batch_end_idx = int(np.min([im_batch_start_idx + batch_size, num_images]))
                batch_images = images[im_batch_start_idx:im_batch_end_idx, ...]
                feed_dict = model.get_feed_dict(batch_images, is_test=True)
                feed_dict[model.weight_placeholder] = weights
                outputs = sess.run(activation_operation(), feed_dict)
                activations.append(outputs.copy())
            activations = np.stack(activations, axis=0)
            num_batches, batch_size, num_outputs = activations.shape
            activations = activations.reshape((num_batches*batch_size, num_outputs))
        else:
            feed_dict = model.get_feed_dict(images, is_test=True)
            feed_dict[model.weight_placeholder] = weights
            sess.run(model.init_op, feed_dict)
            activations = sess.run(activation_operation(), feed_dict)
    return activations

In [ ]:
lamb_activation = lambda x : tf.identity(x) # linear
lambda_params = pp.get_params("lambda")
lambda_params.set_data_params("vanhateren")
lambda_params.batch_size = 100
lambda_params.data_shape = [analysis_params.patch_edge_size**2] # assumes vector inputs (i.e. not convoultional)
lambda_params.activation_function = lamb_activation
num_neurons_list = [analyzer.model_params.num_neurons for analyzer in analyzers]
for num_neurons, lca_weights in zip(num_neurons_list, weights):
    lambda_params.num_neurons = num_neurons
    lambda_model = mp.get_model("lambda")
    lambda_model.setup(lambda_params)
    lambda_activations = compute_lambda_activations(
        data['train'].images[0:100, ...],
        lambda_model,
        lca_weights,
        batch_size=10
    )
    activations.append(lambda_activations)

In [ ]:
def normalize_vector(vector):
    """
    ensure input is a vector, and then divide it by its l2 norm.
    Parameters:
        vector [ np.ndarray] vector with shape [vector_length,].
            It can also be a square image, which will first be vectorized
    Outputs:
        vector [np.ndarray] vector with shape [vector_length,] and l2-norm = 1
    """
    vector = vector.reshape(vector.size)
    vector = vector / np.linalg.norm(vector)
    return vector

def get_paired_vector_angles(vec_list_a, vec_list_b):
    """
    Compute the angle in degrees between all pairs of vectors
    Parameters:
        vec_list_a [list] list of vectors (e.g. images) to compute the angle bewteen
        vec_list_b [list] list of vectors (e.g. images) to compute the angle bewteen
    Outputs:
        angle_matrix [np.ndarray] of shape [num_vectors, num_vectors] with all angles between
            basis functions in the lower triangle and upper triangle is set to -1
    """
    num_vecs_a = len(vec_list_a)
    num_vecs_b = len(vec_list_b)
    vec_length = vec_list_a[0].size
    assert vec_list_a[0].size == vec_length, ("vector lengths must be equal")
    angle_matrix = np.zeros((num_vecs_a, num_vecs_b))
    for a_idx in range(num_vecs_a):
        for b_idx in range(num_vecs_b):
            vec0 = normalize_vector(vec_list_a[a_idx]).reshape((vec_length, 1))
            vec1 = normalize_vector(vec_list_b[b_idx]).reshape((vec_length, 1))
            inner_products = np.dot(vec0.T, vec1)
            inner_products = np.clip(inner_products, -1.0, 1.0)
            angle = np.arccos(inner_products)
            angle_matrix[a_idx, b_idx] = angle * (180 / np.pi)
    return angle_matrix

In [ ]:
image_list = [data['train'].images[idx, ...].reshape((analysis_params.patch_edge_size**2, 1))
    for idx in range(100)]
weight_lists = [[lca_weights[:, idx].reshape((analysis_params.patch_edge_size**2, 1))
    for idx in range(lca_weights.shape[1])]
    for lca_weights in weights]
angle_matrix_list = [get_paired_vector_angles(image_list, weight_list) for weight_list in weight_lists]

In [ ]:
fig, axs = plot.subplots(ncols=len(angle_matrix_list))
for idx in range(len(angle_matrix_list)):
    im = axs[idx].imshow(angle_matrix_list[idx], vmin=0, vmax=180)
    axs[idx].format(title=analyzers[idx].model_label)
axs[-1].colorbar(im, ticks=[0, 90, 180])
axs.format(suptitle='Weight-to-Image Angles')
plot.show()

In [ ]:
neuron_idx = 0 #TODO: repeat this for all neurons
interesting_images = []
for activity in activations:
    interesting_images.append(np.nonzeros(activity[:, neuron_idx] > activity[:, neuron_idx].max()/2))